In [ ]:
#to remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importing all the important libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

In [ ]:
#reading necessary files
names = pd.read_table('mimic_synthetic_feat.name',sep='\n',header=None)
data = pd.read_table('mimic_synthetic_train.data',sep=' ',header=None)
died = pd.read_table('mimic_synthetic_train.solution',sep='\n',header=None)
test = pd.read_table('mimic_synthetic_test.data' ,sep=' ' ,header=None)

In [ ]:
name_col = names[0].tolist() #converting the name row of excel to a list

In [ ]:
data.drop(0,axis=1,inplace=True) 
test.drop(0,axis=1,inplace=True)
data.columns = name_col       #assigning column names
test.columns =name_col

In [ ]:
data["MARITAL_STATUS"].fillna("MARRIED", inplace = True) 
data["ICU"].fillna("MICU", inplace = True) 
data["LANGUAGE"].fillna("ENGL", inplace = True)
test["LANGUAGE"].fillna("ENGL",inplace = True)

In [ ]:
#columns headings to a different lists acc to their datatypes
num_cols = []
cat_col = []

for cols in data.columns:
    if data[cols].dtype == object:
        cat_col.append(cols)
    else:
        num_cols.append(cols)

In [ ]:
#columns headings to a different lists acc to their datatypes
tnum_cols = []
tcat_col = []

for cols in test.columns:
    if test[cols].dtype == object:
        tcat_col.append(cols)
    else:
        tnum_cols.append(cols)

In [ ]:
#filling the missing values with median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
data[num_cols] = imputer.fit_transform(data[num_cols])


In [ ]:
#filling missing values with median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
test[tnum_cols] = imputer.fit_transform(test[tnum_cols])

In [ ]:
#dropping unnecessary columns
data.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)
test.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)

In [ ]:
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

In [ ]:
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

In [ ]:
for name_col in test.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(test[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

In [ ]:
data["ETHNICITY"]= data["ETHNICITY"].replace("UNKNOWN/NOT_SPECIFIED", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("UNABLE_TO_OBTAIN", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("PATIENT_DECLINED_TO_ANSWER", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("AMERICAN_INDIAN/ALASKA_NATIVE_FEDERALLY_RECOGNIZED_TRIBE", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("HISPANIC/LATINO_-_HONDURAN", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("UNKNOWN/NOT_SPECIFIED", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("UNABLE_TO_OBTAIN", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("PATIENT_DECLINED_TO_ANSWER", "OTHER")






In [ ]:
imp_ft = []
nimp_ft = []
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        if(unique_cat>1):
            imp_ft.append(name_col)
            nimp_ft.append(name_col)

In [ ]:
cimp_ft = []
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        imp_ft.append(name_col)
        cimp_ft.append(name_col)

In [ ]:
len(imp_ft)

In [ ]:
fdata=data[imp_ft]

In [ ]:
ftest=test[imp_ft]

In [ ]:
Gender_mapping={'M':0,'F':1}                       # convert string to numeric
fdata['GENDER']=fdata['GENDER'].map(Gender_mapping)
ftest['GENDER']=ftest['GENDER'].map(Gender_mapping)


In [ ]:
fdata.to_csv("mytrain.csv")   #preprocessed training file

In [ ]:
ftest.to_csv("mytest.csv")    #peprocessed test file

In [ ]:
died = pd.read_table('mimic_synthetic_train.solution',sep='\n',header=None)   #solution of training data set

In [ ]:
died.columns = ['DIED']

In [ ]:
fdata = pd.get_dummies(fdata)   #one-hot encoding(for string variables)

In [ ]:
ftest = pd.get_dummies(ftest)    #one-hot encoding

In [ ]:
fdata = pd.concat([fdata,died],axis=1)    #adding died as the last column

In [ ]:
from sklearn import preprocessing
#for training data
x = fdata.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fdata = pd.DataFrame(x_scaled)              #changing the array to a Data Frame

#for test data
x = ftest.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
ftest = pd.DataFrame(x_scaled)

In [ ]:
X = fdata.iloc[:,:-1]         #Train data without output class
Y = fdata.iloc[:,-1:]         #solution of train data

In [ ]:
from sklearn.model_selection import train_test_split            #Split data into train & test data 
x_train_res, x_test, y_train_res, y_test = train_test_split(X, Y , test_size = 0.20, random_state = 100)

In [ ]:
X=pd.DataFrame(x_train_res)
Y=pd.DataFrame(y_train_res)

In [ ]:

from imblearn.over_sampling import SMOTE                        # Balance imbalance data
sm = SMOTE(random_state=2)
X, Y = sm.fit_sample(X, np.ravel(Y))
print(sum(Y==0))
print(sum(Y==1))

In [ ]:
#MODEL TESTING

In [ ]:
#Logistic Regression  
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()                                    #Logistic Regression
logreg.fit(X, Y)
y_pred = logreg.predict(x_test)

acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)      # Find Accuracy
print(acc_logreg)

In [ ]:
ty_pred = logreg.predict(ftest)

In [ ]:
c = 0
for i in ty_pred:
    if i == 1:
        c+=1

In [ ]:
c

In [ ]:
over_log = pd.DataFrame(ty_pred)

In [ ]:
over_log.to_csv("mimic_synthetic_test.csv")   #75.76

In [ ]:
#OTHER MODELS

In [ ]:
# Support Vector Machines
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

In [ ]:
ty_pred = svc.predict(ftest)

In [ ]:
#Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)

y_pred = linear_svc.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

In [ ]:
ty_pred = svc.predict(ftest)

In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train_res, y_train_res)

y_pred = decisiontree.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

In [ ]:
ty_pred = decisiontree.predict(ftest)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest =  RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)
randomforest.fit(x_train_res, y_train_res)

y_pred = randomforest.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)


In [ ]:
ty_pred = randomforest.predict(ftest)

In [ ]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train_res, y_train_res)

y_pred = gbk.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

In [ ]:
ty_pred = gbk.predict(ftest)

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train_res, y_train_res)

y_pred = gaussian.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

In [ ]:
ty_pred = gaussian.predict(ftest)

In [ ]:
# Perceptron
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(x_train_res, y_train_res)
y_pred = perceptron.predict(x_test)
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

In [ ]:
ty_pred = perceptron.predict(ftest)

In [ ]:
c=0
for i in ty_pred:
    if i==1:
        c+=1

In [ ]:
c